In [1]:
import json 
import spacy
import scispacy
from scispacy.linking import EntityLinker
import pubmed_parser as pp
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

dicts_out = pp.parse_medline_xml('../data/pubmed_data/raw_data/44_abstracts_test.xml',
                                 year_info_only=False,
                                 nlm_category=False,
                                 author_list=False,
                                 reference_list=False)

/home/rgoli/.conda/envs/pytorch/lib/python3.8/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_sci_lg' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/rgoli/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rgoli/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 w

In [2]:
dicts_out[0]

{'title': 'ASTI: a guideline-based drug-ordering system for primary care.',
 'abstract': 'Existing computer-based ordering systems for physicians provide effective drug-centered checks but offer little assistance for optimizing the overall patient-centered treatment strategy. Evidence-based clinical practice guidelines have been developed to disseminate state-of-the-art information concerning treatment strategy but these guidelines are poorly used in routine practice. The ASTI project aims to design a guideline-based ordering system to enable general practitioners to avoid prescription errors and to improve compliance with best therapeutic practices. The " critic mode " operates as a background process and corrects the physician\'s prescription on the basis of automatically triggered elementary rules that account for isolated guideline recommendations. The " guided mode " directs the physician to the best treatment by browsing a comprehensive guideline knowledge base represented as a d

In [3]:
len(dicts_out)

42

In [4]:
comb_arr=[]
pmid_arr =[]
for paper in dicts_out:
    comb_arr.append(paper['title']+' '+paper['abstract'])
    pmid_arr.append(paper['pmid'])

In [33]:
documents =[]

doc_id=0
for doc in nlp.pipe(comb_arr):
    # sents = [str(x) for x in doc.sents]
    text = [token.orth_ for token in doc if not token.is_punct | token.is_space]
    # text = [token.orth_ for token in doc if not token.is_space]
    documents.append(TaggedDocument(text, [doc_id]))
    doc_id+=1

/home/rgoli/.conda/envs/pytorch/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/rgoli/.conda/envs/pytorch/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [34]:
documents[0:2]

[TaggedDocument(words=['ASTI', 'a', 'guideline-based', 'drug-ordering', 'system', 'for', 'primary', 'care', 'Existing', 'computer-based', 'ordering', 'systems', 'for', 'physicians', 'provide', 'effective', 'drug-centered', 'checks', 'but', 'offer', 'little', 'assistance', 'for', 'optimizing', 'the', 'overall', 'patient-centered', 'treatment', 'strategy', 'Evidence-based', 'clinical', 'practice', 'guidelines', 'have', 'been', 'developed', 'to', 'disseminate', 'state-of-the-art', 'information', 'concerning', 'treatment', 'strategy', 'but', 'these', 'guidelines', 'are', 'poorly', 'used', 'in', 'routine', 'practice', 'The', 'ASTI', 'project', 'aims', 'to', 'design', 'a', 'guideline-based', 'ordering', 'system', 'to', 'enable', 'general', 'practitioners', 'to', 'avoid', 'prescription', 'errors', 'and', 'to', 'improve', 'compliance', 'with', 'best', 'therapeutic', 'practices', 'The', 'critic', 'mode', 'operates', 'as', 'a', 'background', 'process', 'and', 'corrects', 'the', 'physician', "'s"

In [56]:
# model = Doc2Vec(documents, vector_size=50, window=30, min_count=2, workers=1, epochs=40, seed=42)
model = Doc2Vec(documents, vector_size=128, shrink_windows=True, min_count=2, workers=1, epochs=40, seed=42)

In [66]:
##https://radimrehurek.com/gensim/models/doc2vec.html
vector = model.infer_vector(['Continuous', 'cardiorespiratory', 'monitoring', 'is', 'a', 'dominant', 'source', 'of', 'predictive', 'signal', 'in', 'machine', 'learning', 'for', 'risk', 'stratification', 'and', 'clinical', 'decision', 'support', 'Beaulieu-Jones', 'and', 'coworkers', 'propose', 'a', 'litmus', 'test', 'for', 'the', 'field', 'of', 'predictive', 'analytics-performance', 'improvements', 'must', 'be', 'demonstrated', 'to', 'be', 'the', 'result', 'of', 'non-clinician-initiated', 'data', 'otherwise', 'there', 'should', 'be', 'caution', 'in', 'assuming', 'that', 'predictive', 'models', 'could', 'improve', 'clinical', 'decision-making', 'Beaulieu-Joneset', 'al2021', 'They', 'demonstrate', 'substantial', 'prognostic', 'information', 'in', 'unsorted', 'physician', 'orders', 'made', 'before', 'the', 'first', 'midnight', 'of', 'hospital', 'admission', 'and', 'we', 'are', 'persuaded', 'that', 'it', 'is', 'fair', 'to', 'ask-if', 'the', 'physician', 'thought', 'of', 'it', 'first', 'what', 'exactly', 'is', 'machine', 'learning', 'for', 'in-patient', 'risk', 'stratification', 'learning', 'about', 'While', 'we', 'want', 'predictive', 'analytics', 'to', 'represent', 'the', 'leading', 'indicators', 'of', 'a', 'patient', "'s", 'illness', 'does', 'it', 'instead', 'merely', 'reflect', 'the', 'lagging', 'indicators', 'of', 'clinicians', 'actions', 'We', 'propose', 'that', 'continuous', 'cardiorespiratory', "monitoring-'routine", 'telemetry', 'data', 'in', 'Beaulieu-Jones', 'terms-represents', 'the', 'most', 'valuable', 'non-clinician-initiated', 'predictive', 'signal', 'present', 'in', 'patient', 'data', 'and', 'the', 'value', 'added', 'to', 'patient', 'care', 'justifies', 'the', 'efforts', 'and', 'expense', 'required', 'Here', 'we', 'present', 'a', 'clinical', 'and', 'a', 'physiological', 'point', 'of', 'view', 'to', 'support', 'our', 'contention'])
vector

array([-0.33529308,  0.00414462,  0.34992164,  0.01153625, -0.13112226,
        0.2744652 ,  0.16004454, -0.26437718, -0.07256277, -0.25427637,
        0.55504876, -0.4460526 ,  0.44256127, -0.44208002,  0.01014696,
        0.43693516, -0.504732  ,  0.02661002, -0.42967793, -0.02594503,
        0.11093866,  0.76053536,  0.10422097,  0.14614084,  0.3729742 ,
        0.326285  ,  0.00145865, -0.06264268,  0.16383515, -0.15524615,
        0.25348425, -0.3862859 ,  0.2309519 , -0.6686532 ,  0.488442  ,
       -0.2353975 ,  0.14880599,  0.6815336 , -0.00150042, -0.38059917,
       -0.36264238,  0.7384407 ,  0.15221977,  0.6478087 ,  0.27911428,
        0.13441204, -0.08679418,  0.22578771, -0.1164505 ,  0.12677194,
       -0.46965516, -0.34450796,  0.20098211, -0.28952408, -0.15831755,
       -0.32604113,  0.22450778,  0.03892839,  0.30745432, -0.02661484,
        0.1651819 , -0.2671483 ,  0.7422471 ,  0.16411254, -0.26512545,
       -0.02638624, -0.17873889,  0.09976531, -0.01144623, -0.03